In [1]:
import geopandas as gpd
import pandas as pd
from utils import gdf_operations, functions

2025-01-29 16:03:40.258 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
distrito =gdf_operations.get_dados('distrito')
subbac = gdf_operations.get_dados('subbac')
subpref = gdf_operations.get_dados('subpref')
fcu = gdf_operations.get_dados('fcu')

In [3]:
cd_subbac = gdf_operations.find_subbac_name(subbac, 'cd_')
nm_subbac = gdf_operations.find_subbac_name(subbac, 'nm_')

unidades_list = [
    ('subbac', "Sub Bacias Hidrográficas", cd_subbac, nm_subbac),
    ('subpref', "Subprefeituras", "cd_subpref" , 'nm_subpref'),
    ('distrito', "Distritos", 'cd_distrit', 'nm_distrit'),
    ('fcu', "Favelas e Comunidades Urbanas", 'cd_fcu', 'nm_fcu')
    
]
unidades_df = pd.DataFrame(unidades_list, columns=['gdf_name', 'name', 'column_cd', 'column_name'])


Vou começar pelo fcu para ver se dá certo.

# FCU

In [4]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [5]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
fcu
column_name


overlay_gdf = gdf_operations.overlay_intersec(
    'distrito', gdf_unidade, 'cd_fcu')

# Distritos

Agora vamos tentar com os distritos

In [6]:
distrito_sorted = gdf_operations.create_gdf_sorted(
    distrito, 'distrito'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [7]:
choice_unidade = 'distrito'
gdf_unidade = distrito_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
distrito
column_name


In [8]:
overlay_gdf = gdf_operations.overlay_intersec(
    'subpref', gdf_unidade, 'cd_distrit'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


# Loop por intersec

## Vamos Começar com o FCU mesmo

In [9]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [10]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

gdf_name
fcu
column_name


In [11]:
print(unidades_df[unidades_df['gdf_name']=='fcu'])

  gdf_name                           name column_cd column_name
3      fcu  Favelas e Comunidades Urbanas    cd_fcu      nm_fcu


In [12]:
index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == choice_unidade
        ].index[0]
    )

name_gdf_unidade = choice_unidade
batatinha = pd.DataFrame()

cd_unidade = gdf_operations.find_gdf_name(gdf_unidade, choice_unidade, 'cd_')

overlay_gdf = gpd.GeoDataFrame()


for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )
            print(overlay_gdf.columns)
            
            gdf_unidade[cd_unidade] = gdf_unidade[cd_unidade].astype(str)
            overlay_gdf[cd_intersec] =(
                overlay_gdf[cd_intersec]
                .astype(int)
                .astype(str)
            )
            
            overlay_gdf[cd_unidade] = (
                overlay_gdf[cd_unidade]
                .astype(str)
            )


            gdf_unidade = pd.merge(gdf_unidade, overlay_gdf[[cd_unidade, cd_intersec]], on=cd_unidade, how='left')


gdf_operations.save_intersec(gdf_unidade, choice_unidade)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'nm_bacia_h', 'geometry', 'area'],
      dtype='object')


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'cd_subpref', 'nm_subpref', 'geometry', 'area'],
      dtype='object')


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'cd_subpref', 'cd_distrit', 'nm_distrit', 'geometry', 'area'],
      dtype='object')


In [13]:
gdf_unidade.sample(3)

,cd_fcu,nm_fcu,pop_fcu,dom_fcu,con_fcu,geometry,cd_identif,cd_subpref,cd_distrit
275,35503080936,Santa Etelvina III,451,137,63140,"POLYGON ((356871.615 7390694.732, 356858.753 7...",706,31,25
366,35503082113,São João da Boa Vista II,253,109,35420,"POLYGON ((331941.1 7380588.291, 331946.484 738...",582,16,22
2,35503081646,Pavanas,161,58,22540,"POLYGON ((331961.694 7381437.08, 331927.976 73...",582,16,22


## Intersec distrito

In [14]:
distrito_sorted = gdf_operations.create_gdf_sorted(
    distrito, 'distrito'
)
choice_unidade = 'distrito'
gdf_unidade = distrito_sorted
name_column_unidade = functions.find_gdf_info(
    unidades_df, 
    choice_unidade, 
    'gdf_name', 
    'column_name'
)

gdf_name
distrito
column_name


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [15]:
index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == choice_unidade
        ].index[0]
    )

name_gdf_unidade = choice_unidade
batatinha = pd.DataFrame()

cd_unidade = gdf_operations.find_gdf_name(gdf_unidade, choice_unidade, 'cd_')

overlay_gdf = gpd.GeoDataFrame()


for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )
            print(overlay_gdf.columns)
            
            gdf_unidade[cd_unidade] = gdf_unidade[cd_unidade].astype(str)
            overlay_gdf[cd_intersec] = overlay_gdf[cd_intersec].astype(int).astype(str)
            overlay_gdf[cd_unidade] = overlay_gdf[cd_unidade].astype(str)


            gdf_unidade = pd.merge(gdf_unidade, overlay_gdf[[cd_unidade, cd_intersec]], on=cd_unidade, how='left')


gdf_operations.save_intersec(gdf_unidade, choice_unidade)


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_distrit', 'nm_distrit', 'pop_total', 'dom_total', 'consumo_di',
       'cd_identif', 'nm_bacia_h', 'geometry', 'area'],
      dtype='object')


D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_distrit', 'nm_distrit', 'pop_total', 'dom_total', 'consumo_di',
       'cd_identif', 'cd_subpref', 'nm_subpref', 'geometry', 'area'],
      dtype='object')


In [16]:
gdf_unidade.sample(3)

,cd_distrit,nm_distrit,pop_total,dom_total,consumo_di,geometry,cd_identif,cd_subpref
28,31,GUAIANASES,109966,44162,15395240,"POLYGON ((355108.237 7393291.184, 355107.277 7...",706,28
8,46,JARDIM SAO LUIS,260203,106144,36428420,"POLYGON ((321628.005 7377780.623, 321626.439 7...",570,18
63,1,AGUA RASA,86130,41340,12058200,"POLYGON ((341093.38 7391912.491, 341093.329 73...",641,25


## Testar DE NOVO O LOOP

In [17]:
intersec_fcu = gdf_operations.intersec_unidades(unidades_df, fcu, 'fcu')
intersec_fcu.sample(3)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]
D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geo

,cd_fcu,nm_fcu,pop_fcu,dom_fcu,con_fcu,geometry,cd_identif,cd_subpref,cd_distrit
494,35503082185,Alba,1213,466,169820,"POLYGON ((330839.545 7384299.194, 330865.703 7...",585,15,38
638,35503081276,Parque Cavalo Branco,3922,1448,549080,"POLYGON ((318882.503 7373159.938, 318875.944 7...",544,18,43
969,35503080520,Jardim Sônia Ingá,1134,457,158760,"POLYGON ((319735.314 7380410.926, 319727.993 7...",581,17,19


In [18]:
intersec_distrito = gdf_operations.intersec_unidades(unidades_df, distrito, 'distrito')
intersec_distrito.sample(3)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]
D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geo

,cd_distrit,nm_distrit,pop_total,dom_total,consumo_di,geometry,cd_identif,cd_subpref
12,49,LIBERDADE,65908,37809,9227120,"POLYGON ((332502.702 7392061.729, 332502.811 7...",623,9
22,76,SAPOPEMBA,266411,105476,37297540,"POLYGON ((344303.158 7389068.033, 344303.09 73...",645,32
66,4,ARICANDUVA,90894,37830,12725160,"POLYGON ((344068.781 7390960.855, 344068.149 7...",645,26


In [19]:
intersec_subpref = gdf_operations.intersec_unidades(
    unidades_df, subpref, 'subpref')
intersec_subpref.sample(3)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]
D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:122: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geo

,cd_subpref,nm_subpref,pop_total,dom_total,consumo_di,geometry,cd_identif
3,07,VILA MARIA-VILA GUILHERME,276057,120169,38647980,"POLYGON ((336675.044 7401136.914, 336698.453 7...",693
23,13,IPIRANGA,480163,213546,67222820,"POLYGON ((335287.875 7391129.763, 335279.12 73...",615
9,09,SE,424112,258563,59375680,"POLYGON ((330175.535 7396046.238, 330181.745 7...",636


In [20]:
intersec_subbac = gdf_operations.intersec_unidades(
    unidades_df, subbac, 'subbac')
intersec_subbac.sample(3)

D:\Documentos_Novo\ADPPSP\2024\Saneamento básico\PMSB\hydra\streamlit-pmsb\utils\gdf_operations.py:133: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


,cd_identif,nm_bacia_h,pop_total,dom_total,consumo_di,geometry
200,688.0,Córrego Tiquatira - Leste,261809,110677,36653260,"POLYGON ((347832.117 7400378.699, 347842.277 7..."
63,560.0,Córrego Apucas,106614,41517,14925960,"POLYGON ((332266.043 7379880.81, 332266.146 73..."
70,567.0,Área de Contribuição Direta de Escoamento Difu...,26865,10953,3761100,"POLYGON ((323817.709 7381071.618, 323810.422 7..."
